In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (SparkSession.builder
         .config("spark.jars.packages", "org.postgresql:postgresql:42.3.3")
         .getOrCreate()
        )

In [ ]:
host = "postgres"
port = "5432"
user = "myuser"
pw = "MyPassw0rd!" # never show your password 
db = "visual"
schema = "public"
tbl = "stream_loc" # you need to create this table on postgres beforehand. 

In [ ]:
schema = (spark.read
             .parquet("output/kafka_us.parquet/*.parquet")
            ).schema

kafka_df = (spark.readStream
     .format("parquet")
     .schema(schema)
     .option("path", "output/kafka_us.parquet/*.parquet")
     .load()
)


In [ ]:
json_schema = """
STRUCT<gender: STRING,
name: STRUCT<title: STRING,
            first: STRING,
            last: STRING>,
location: STRUCT<street: STRUCT<number: INT,
                                name: STRING>,
                 city: STRING,
                state: STRING,
                country: STRING,
                postcode: INT,
                coordinates: STRUCT<latitude: STRING,
                                    longitude: STRING>,
                timezone: STRUCT<offset: STRING,
                                description: STRING>
                >,
email: STRING,
login: STRUCT< uuid: STRING,
            username: STRING,
            password: STRING,
            salt: STRING,
            md5: STRING,
            sha1: STRING,
            sha256: STRING>,
dob: STRUCT<date: STRING,
            age: INT>,
registered: STRUCT<date: STRING,
                    age: INT>,
phone: STRING,
cell: STRING,
id: STRUCT<name: STRING,
            value: STRING>,
picture: STRUCT<large: STRING,
                medium: STRING,
                thumbnail: STRING>,
nat: STRING,
timestamp: STRING>
"""

In [ ]:
kafka_df = (kafka_df
    .select(F.from_json(F.col("value").cast("string"), json_schema).alias("json"),
            F.col("timestamp").alias("ts")))


In [ ]:
kafka_df.createOrReplaceTempView("kafka")


In [ ]:
df = spark.sql("""
SELECT CONCAT(json.name.first, ' ', json.name.last) as name,
CAST(json.location.coordinates.latitude AS float) as latitude,
CAST(json.location.coordinates.longitude AS float) as longitude
FROM kafka
""")


In [ ]:
def foreach_batch_function(df_batch, epoch_id):
    (df_batch.write.format("jdbc") 
        .option("driver", 'org.postgresql.Driver')
        .option("url", f"jdbc:postgresql://{host}:{port}/{db}") 
        .option("dbtable", f"{tbl}") 
        .option("user", user) 
        .option("password", pw) 
        .mode("append")
        .save()
    )

In [ ]:
q = df.writeStream.outputMode("append").foreachBatch(foreach_batch_function).start() 

In [ ]:
q.awaitTermination()